In [1]:
import feather as ft
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
date = '2020-03-10'
nextDate = pd.datetime.strftime(pd.datetime.strptime(date, '%Y-%m-%d') + pd.Timedelta('1 days'), '%Y-%m-%d')

In [3]:
path = '../../share/data_prod/'
pathMemberWallet = '../../share/data_prod/MemberWallets/'
pathMemberAccountLog = '../../share/data_prod/MemberAccountLog/'

In [9]:
tmp = ft.read_dataframe('../../share/data_prod/revenueError.ft')

In [13]:
MemberAccount.query('MemberId == 156290 & AddTime >= "2020-03-10"')

,LId,MemberId,MemberName,OrderId,Currency,CurrentBalance,OldBalance,Amount,Type,AddTime,Remark
283006,44954853,156290,328039393,PG2003100942dp4ww,RMB,1636431.0,368322.49,1268108.51,payment-grantPayout,2020-03-10 09:42:57.113,
288967,44960814,156290,328039393,ADJ2003101445h4elg,RMB,0.0,1636431.00,-1636431.00,Payment-Adjustment,2020-03-10 14:45:43.413,代理费手动出


In [18]:
path = '../../share/data_prod/'
tmp.drop(100).reset_index(drop=True).to_feather(path + 'revenueError.ft')

In [14]:
tmp.query('日期 == "2020-03-10"')

,用戶ID,用戶名稱,存款,提款,彩金＋返水,地方錢包餘額,輸贏（公司）,中心錢包原有額度,存提差,輸贏與存提差,日期
0,156290,328039393,0.0,-1636431.0,0.00,0.050000,0.00,368322.49,-1636431.0,-1636431.00,2020-03-10
1,482127,dks15205511395,935000.0,0.0,182441.55,502072.500000,1379413.40,0.34,935000.0,-444413.40,2020-03-10
2,157170,gzlovehate,43000.0,-357186.0,0.00,459.550000,-73638.48,240088.27,-314186.0,-240547.52,2020-03-10
3,14000,xfcctv,33300.0,0.0,5876.39,214721.760000,243249.48,0.31,33300.0,-209949.48,2020-03-10
4,231309,yuzhiyu520,386796.0,0.0,23864.59,40473.000000,550731.15,128000.52,386796.0,-163935.15,2020-03-10
...,...,...,...,...,...,...,...,...,...,...,...
96,300592,dada0930,10400.0,0.0,2121.53,1611.490000,18985.67,6475.34,10400.0,-8585.67,2020-03-10
97,507571,yqq123456789,1540.0,-7500.0,74.68,1594.000000,2589.97,7767.04,-5960.0,-8549.97,2020-03-10
98,106267,gy59761376,2500.0,0.0,858.61,7688.660000,11046.16,0.00,2500.0,-8546.16,2020-03-10
99,173570,redblack,2000.0,-8500.0,0.00,9999.996586,1660.00,0.00,-6500.0,-8160.00,2020-03-10


In [4]:
# 地方錢包金額
MemberWallets = ft.read_dataframe(pathMemberWallet + 'MemberWalletsFlow' + nextDate.replace('-', '') + '.ft')

# 中心錢包金額
# 根據目標月份，取得當月中心錢包變化資料
MemberAccount = pd.DataFrame()
MemberAccountList = os.listdir(pathMemberAccountLog)
for name in MemberAccountList:
    if '.ft' in name:        
        # 2020年02月後才需要，地方錢包金額是從 2020/2/19 之後開始打
        if (int(name[-5:-3]) > 1) & (int(name[-9:-5]) >= 2020):
            MemberAccount = MemberAccount.append(ft.read_dataframe(pathMemberAccountLog + name))

# 流水
gameSummary = ft.read_dataframe(path + 'game_daily_summary.ft')
member_profile = ft.read_dataframe(path + 'member_profile.ft')

In [5]:
def renameFuc(df, colOldName, colNewName):
    return df.rename(columns={colOldName:colNewName})

In [6]:
# 當天中心錢包變化資料
df = MemberAccount.query('AddTime >= @date & AddTime < @nextDate')
df.Type = df.Type.str.lower()

In [7]:
# 存
MemberDepWitFlow = df.query('Type.isin(["thirdpayment", "payment-adjustment", "payment to deposit"])').groupby('MemberId').Amount.sum().reset_index()
MemberDepWitFlow = renameFuc(MemberDepWitFlow, 'Amount', 'Deposit')
# 提
MemberDepWitFlow = MemberDepWitFlow.merge(df.query('Type == "payment-withdrawal"').groupby('MemberId').Amount.sum().reset_index(), on = 'MemberId', how = 'outer')
MemberDepWitFlow = renameFuc(MemberDepWitFlow, 'Amount', 'Withdrawal')
# 其他
MemberDepWitFlow = MemberDepWitFlow.merge(df.query('Type.isin(["affiliatewallettrans", "payment-batchadjuest"])').groupby('MemberId').Amount.sum().reset_index(), on = 'MemberId', how = 'outer')
MemberDepWitFlow = renameFuc(MemberDepWitFlow, 'Amount', 'batchAdjuest')
# 地方錢包金額
localSum = MemberWallets.query('Balance > 0').groupby('MemberId').Balance.sum().reset_index()
localSum = renameFuc(localSum, 'Balance', 'localSum')
MemberDepWitFlow = MemberDepWitFlow.merge(localSum, on = 'MemberId', how = 'outer')
# 輸贏
winLoss = gameSummary.query('date == @date').groupby('member_id').gross_revenue_payout_time.sum().reset_index()
winLoss = renameFuc(winLoss, 'gross_revenue_payout_time', 'winLoss')
winLoss = renameFuc(winLoss, 'member_id', 'MemberId')
MemberDepWitFlow = MemberDepWitFlow.merge(winLoss, on = 'MemberId', how = 'outer')
# 原始中心錢包金額
# 中心錢包變動紀錄
MemberAccount['targetDate'] = pd.datetime.strptime(date, '%Y-%m-%d')
MemberAccount['diff'] = abs(MemberAccount['targetDate'] - MemberAccount['AddTime'])
MemberAccount['中心錢包餘額'] = MemberAccount['OldBalance']
MemberAccount.loc[MemberAccount.AddTime < date, '中心錢包餘額'] = MemberAccount.loc[MemberAccount.AddTime < date, 'CurrentBalance']
closestLog = MemberAccount.groupby('MemberId')['diff'].min().reset_index()
closestLog['isClosest'] = 1
MemberAccount = MemberAccount.merge(closestLog, on = ['MemberId', 'diff'], how = 'left')
startCenterAccount = MemberAccount.query('isClosest == 1').groupby('MemberId')['中心錢包餘額'].min().reset_index()
MemberDepWitFlow = MemberDepWitFlow.merge(startCenterAccount, on = 'MemberId', how = 'outer')



In [8]:
MemberDepWitFlow = MemberDepWitFlow.fillna(0)
MemberDepWitFlow['depWitDiff'] = MemberDepWitFlow.Deposit + MemberDepWitFlow.Withdrawal 
MemberDepWitFlow['dwWinLossDiff'] = MemberDepWitFlow.depWitDiff - MemberDepWitFlow.winLoss
member_profile['MemberId'] = member_profile['member_id']

In [9]:
MemberDepWitFlow = MemberDepWitFlow.sort_values(by = 'dwWinLossDiff').head(100).merge(member_profile[['MemberId', 'member_name']], on = 'MemberId', how = 'left').reset_index(drop=True)

In [10]:
MemberDepWitFlow = MemberDepWitFlow[['MemberId', 'member_name', 'Deposit', 'Withdrawal', 'batchAdjuest', 'localSum',
       'winLoss', '中心錢包餘額', 'depWitDiff', 'dwWinLossDiff']].rename(columns={
    'MemberId':'用戶ID', 'member_name':'用戶名稱', 'Deposit':'存款', 'Withdrawal':'提款', 
    'batchAdjuest':'彩金＋返水', 'localSum':'地方錢包餘額',
       'winLoss':'輸贏（公司）', '中心錢包餘額':'中心錢包原有額度', 'depWitDiff':'存提差', 'dwWinLossDiff':'輸贏與存提差'
})

In [11]:
MemberDepWitFlow['日期'] = pd.datetime.strptime(date, '%Y-%m-%d')

In [16]:
revenueError = ft.read_dataframe(path + 'revenueError.ft')
MemberDepWitFlow.append(revenueError).drop_duplicates().reset_index(drop = True).to_feather(path + 'revenueError.ft')